http://jira.lge.com/issue/browse/HEDATAPLFM-1040?attachmentSortBy=dateTime&attachmentOrder=asc


In [0]:
df = spark.read.csv(
    "/Volumes/sandbox/z_najin_lee/v_najin_lee/lgch_rec_segment_0925.csv",
    header=True,
    inferSchema=True,
)

df.show(10)

In [0]:
from pyspark.sql.functions import (
    sha2,
    concat_ws,
    current_timestamp,
    when,
    col,
    lit,
    sha2,
    concat,
)


he_etl_dt = current_timestamp()
tv_salt = dbutils.secrets.get("admin", "salt")


# df_private 컬럼 추가
df = df.withColumn(
    "mac_addr",
    when(col("mac").isNull() | (col("mac") == ""), col("mac")).otherwise(
        sha2(concat(col("mac"), lit(tv_salt)), 256)
    ),
)

df = df.withColumn("he_etl_dt", he_etl_dt)

display(df)

In [0]:
df = df.drop("mac")
display(df)

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("sandbox.z_najin_lee.lgch_rec_segment_0925")

In [0]:
%sql
select *
from sandbox.z_najin_lee.lgch_rec_segment_0925
limit 20;

In [0]:
%sql
select distinct mac
from sandbox.z_najin_lee.lgch_rec_segment_0925
limit 20;